In [3]:
#from matplotlib.cbook.deprecation import contextlib
import pandas as pd
import numpy as np
from math import pi
from math import isnan
import cv2
import os
from sklearn import linear_model, datasets
from matplotlib import pyplot as plt
import statistics
import scipy
import csv
import re
import sys

#ANGLE_REJECTION = pi/6
#RANSAC_REJECTION = 150
#MODE = 'mean'
#DISTANCE = 18

### parameterization
# pi/9 190 median 36

# conf .005 params
#rej_angle = pi/5
#		ransac_rej = 170
#		rej_mode = 'mode'
#		rej_distance = 18

def error(ground_truth, ransac_table):
	err = 0
	keys = ransac_table['image'].unique()
	keys2 = ground_truth['fibril'].unique()
	keys3 = [k for k in keys if k[:-4] in keys2]
	for i in keys3:
		gt = ground_truth.loc[ground_truth['fibril'] == i[:-4]]['xover'].to_list()[0]
		if isinstance(gt, str):
			gt = gt.replace(',','.')
			gt = float(gt)
		ra = float(ransac_table.loc[ransac_table['image'] == i]['xover'].to_list()[0])
		if not (isnan(gt) or isnan(ra)):
			err += (gt - ra)**2
	return err

def group(row):
    if bool(re.search("wt42_seeding_wt", row['image'])):
        return("wt42_seeding_wt")
    elif bool(re.search("SBA_PEG", row['image'])):
        return("SBA_PEG")
    elif bool(re.search("SBA_PFDTES", row['image'])):
        return("SBA_PFDTES")
    elif bool(re.search("[a-f][0-9]+_arctic_[0-9]{4}", row['image'])):
        return("arctic")
    elif bool(re.search("osaka_seeding_wt42", row['image'])):
        return("osaka_seeding_wt42")
    elif bool(re.search("wt40_seeding_wt42", row['image'])):
        return("wt40_seeding_wt42")
    elif bool(re.search("osaka_seeding_wt", row['image'])):
        return("osaka_seeding_wt")
    elif bool(re.search("[a-f][0-9]+_osaka_[0-9]{4}", row['image'])):
        return("osaka")
    elif bool(re.search("arctic_seeding_wt", row['image'])):
        return("arctic_seeding_wt")
    elif bool(re.search("wt_seeding_wt_gen2", row['image'])):
        return("wt_seeding_wt_gen2")
    elif bool(re.search("wt_seeding_wt_gen3", row['image'])):
        return("wt_seeding_wt_gen3")
    elif bool(re.search("wt_seeding_osaka", row['image'])):
        return("wt_seeding_osaka")
    elif bool(re.search("wt_seeding_wt", row['image'])):
        return("wt_seeding_wt")
    elif bool(re.search("[a-f][0-9]+_wt42_[0-9]{4}", row['image'])):
        return("wt42")
    elif bool(re.search("wt42_seeding_osaka", row['image'])):
        return("wt42_seeding_osaka")
    elif bool(re.search("wt_seeding_arctic", row['image'])):
        return("wt_seeding_arctic")
    elif bool(re.search("Ab_Ctrl", row['image'])):
        return("Ab_Ctrl")
    elif bool(re.search("Ab_SDS", row['image'])):
        return("Ab_SDS")
    elif bool(re.search("Ab_SiNP", row['image'])):
        return("Ab_SiNP")
    elif bool(re.search("SBA_Ctrl", row['image'])):
        return("SBA_Ctrl")
    print(row['image'])
    raise Exception('uh oh! somebody fucked up grouping!')

def process_xl(dfr):
	dfr['cent_x'] = (dfr.xmin + dfr.xmax)/2
	dfr['cent_y'] = (dfr.ymin + dfr.ymax)/2
	return dfr

def find_zeroes(files):
	xovers = []
	image_path = '/content/drive/MyDrive/hjb/TrainYourOwnYOLO/Data/Source_Images/Training_Images/vott-csv-export/'
	images = os.listdir(image_path)
	zero_images = [x for x in images if x not in files] # thsnk you stackoverflow
	zero_images = [x for x in zero_images if x[-8:-4].isdigit()]
	for i in zero_images:
		if os.path.exists(image_path + "/" + i):
			# copy of cv2 code here
			img = cv2.imread(image_path + "/" + i)
			gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
			edges = cv2.Canny(gray,50,150,apertureSize = 3)
			lines = cv2.HoughLines(edges,1,np.pi/180,200)
			if len(lines) > 0:
				xovers.append(0)
#				print(i)
#				print(0)
	return zero_images, xovers

def lines(x, y, rej_mode, ransac_rej, rej_angle, prob, ranmax, img):
	xline = []
	yline = []
	remainder = []
	xremainder = []
	yremainder = []
	linebounds = []
	ransac = linear_model.RANSACRegressor(max_trials = ranmax, stop_probability = prob)
	for i in np.arange(0,len(x)-3):
		rho = np.tan((y[i] - y[i+1])/(x[i] - x[i+1] + 0.001))
		if rho < rej_angle or rho > -(rej_angle):
			xline.append(x[i])
			yline.append(y[i])
		else:
			remainder.append(x[i], y[i])
	if xline is not None and len(xline) > 2:
		#print("1!")
		xline = np.array(xline)[:, np.newaxis]
		yline = np.array(yline)[:, np.newaxis]
		#print(xline)
		#print(yline)
		ransac.fit(xline,yline)
		x_min = min(xline)
		y_min = ransac.predict(x_min.reshape(1, -1))
		x_max = max(xline)
		y_max = ransac.predict(x_max.reshape(1, -1))
		dist = np.sqrt((x_max-x_min)**2 + (y_max-y_min)**2)
		cnt = 0
		comb = sorted(zip(xline, yline), key=lambda x:x[1])
		comb_nooutliers = []
		for x,y in comb:
			if abs(y - ransac.predict(x.reshape(1, -1))) < ransac_rej:
				comb_nooutliers.append((x,y))
		ranges = []
		for i in range(0, len(comb_nooutliers) - 1, 1):
				x0, y0 = comb_nooutliers[i]
				x1, y1 = comb_nooutliers[i+1]
				dist = int(np.sqrt((x1-x0)**2 + (y1-y0)**2))
				ranges.append(dist)
#		print(ranges)
		if ranges != []:
			if rej_mode == 'average':
				dist = statistics.mean(list(ranges))
			elif rej_mode == 'mode':
				dist = statistics.median(list(ranges))
		else:
			 dist = 999
		cnt = 1
		linebounds.append([(x_min, y_min), (x_max, y_max), (dist, cnt)])
		xover = (dist/cnt) * 0.36
		imgpath = plt.imread('/content/drive/MyDrive/hjb/TrainYourOwnYOLO/Data/Source_Images/Training_Images/vott-csv-export/' + img)
		fig, ax = plt.subplots()
		ax.imshow(imgpath)
		uzx, uzy = [[*x] for x in zip(*comb_nooutliers)]
		uzx = [item for sub_list in uzx for item in sub_list]
		uzy = [item for sub_list in uzy for item in sub_list]
		ax.plot(uzx, uzy, 'b+')
		plt.title(img + "; xover: " + str(xover))
		print(uzx)
		print(uzy)
		ax.scatter(xline, yline, c='r', alpha=0.3, edgecolor='black')
		fig.show()
		#plt.close(fig)
		#input()
	elif remainder is not None and len(remainder) > 2:
		#print("2!")
		xline = remainder[:][0]
		yline = remainder[:][1]
		xline.reshape(1,-1)
		yline.reshape(1,-1)
		ransac.fit(xline,yline)
		x_min = min(xline)
		y_min = ransac.predict(x_min)
		x_max = max(xline)
		y_max = ransac.predict(x_max)
		dist = np.sqrt((x_max-x_min)**2 + (y_max-y_min)**2)
		linebounds.append([(x_min, y_min), (x_max, y_max), (dist, len(xline))])
	return linebounds


def main(params):
	NUM_PARAMS = 6
	if not (len(params) == NUM_PARAMS):
		rej_angle = pi/5
		ransac_rej = 170
		rej_mode = 'mode'
		rej_distance = 18
		prob = 0.99
		ranmax = 100
	else:
		rej_angle = pi/params[0]
		ransac_rej = params[1]
		rej_mode = params[2]
		rej_distance = params[3]
		prob = params[4]
		ranmax = params[5]

	excelpath = '/content/drive/MyDrive/hjb/TrainYourOwnYOLO/Data/Source_Images/Test_Image_Detection_Results/Detection_Results.csv'
	gtruth = '/content/drive/MyDrive/hjb/TrainYourOwnYOLO/gtr.csv'

	ml_output = pd.read_csv(excelpath)
	ml_output = process_xl(ml_output)

	files = ml_output['image'].tolist()
	files = list(set(files))

	listcross = []
	llb = []
	name = ''
	fl_o_or_a = [f for f in files if "_osaka_" in f or "_arctic_" in f]
	for fl in files:
	#	print(x)
		name = fl
		df = ml_output[ml_output['image'] == fl]
		x = list(df.cent_x)
		y = list(df.cent_y)
		r = list(zip(x,y))
		for i in r:
			for j in r:
				distance = ((i[0]-j[0])**2+(i[1]-j[1])**2)**0.5
				#print(distance)
				if i != j and distance < rej_distance:
					r.remove(j)
		x,y = list(zip(*r))
		linelist = lines(x,y, rej_mode, ransac_rej, rej_angle, prob, ranmax, fl)
		for j in linelist:
			crossover = (j[2][0] / j[2][1]) * 0.36
			listcross.append(crossover)
	#		print(crossover)
			llb.append([name,crossover])

	#print(listcross)
	#ze, fe = find_zeroes(files)

	#print(len(llb))

	#ze = zip(ze, [0]*len(ze))
	with open('12epoch_ransac_mod_halved_modal.csv', 'w', newline='') as f:
			w = csv.writer(f)
			w.writerows(llb)
	#		w.writerows(ze)

	labels = ['image', 'xover']
	combined_data = pd.DataFrame(llb, columns=labels)
	combined_data['group'] = combined_data.apply(lambda row: group(row), axis=1)
	combined_data['group'] = combined_data['group'].astype('category')
	print(combined_data.groupby('group')['xover'].mean())
	combined_data.to_csv("/content/drive/MyDrive/hjb/TrainYourOwnYOLO/12epoch_ransac_001conf"+str(params)+".csv")

	gtruth_data = pd.read_csv(gtruth)

	#gt_table = #import csv here
	return error(gtruth_data, combined_data)

if __name__ == "__main__":
	main(sys.argv[1:])

KeyboardInterrupt: ignored

In [6]:
#rej_angle = pi/6
#ransac_rej = 150
#rej_mode = 'average'
#rej_distance = 18

##### remember that this is excluding zeroes

#%%capture cap --no-stderr

angles = range(3,12)
rej = range(50, 200, 10)
mode = ['average', 'mode']
distance = range(18,72,9)
probs = np.arange(0.5,1,0.05)
ranmax = range(50,400,10)

with open('log.txt','w') as te:
  for a in angles:
   for r in rej:
     for m in mode:
        for d in distance:
            for p in probs:
              for x in ranmax:
                print("angle: " + str(a) + "; rej: " + str(r) + "; mode: " + m + "; dist: " + str(d) + "; prob: " + str(p) + "; max: " + str(x))
                strout1 = main([pi/a,r,m,d,p,x])
                print(strout1)
                te.write("angle: " + str(a) + "; rej: " + str(r) + "; mode: " + m + "; dist: " + str(d) + "; prob: " + str(p) + "; max: " + str(x) + "; error: " + str(strout1) + "\n")

#combined_data['group'] = combined_data.apply(lambda row: group(row), axis=1)
#te = open('log.txt','w')
#te.write(cap)

angle: 3; rej: 50; mode: average; dist: 18; prob: 0.5; max: 50


OSError: ignored

In [ ]:
with open('/content/drive/MyDrive/hjb/TrainYourOwnYOLO/Data/Source_Images/Test_Image_Detection_Results/Detection_Results.csv') as f:
  print(str(f))


<_io.TextIOWrapper name='/content/drive/MyDrive/hjb/TrainYourOwnYOLO/Data/Source_Images/Test_Image_Detection_Results/Detection_Results.csv' mode='r' encoding='UTF-8'>


In [ ]:
main([pi/5, 170, 'mode', 18])

ValueError: ignored